# Infraestructure Manager revenue maximization with GSA

## 0. Load libraries

In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np

from benchmarks.railway import Solution, RevenueMaximization
from src.entities import GSA

from typing import List, Mapping, Union

## 1. Define corridor

As an example, we will use the Spanish south high-speed railway corridor.



In [2]:
# Define corridor

corridor = {"MAD": {
                "CIU": {
                    "COR": {
                        "SEV": {
                            "CAD": {}
                        },
                        "PGE": {
                            "ANT": {
                                "GRA": {},
                                "MAL": {}
                                    }
                                }
                            }
                        }
                    }
            }

### 1.1. Get lines from corridor

In [3]:
# Get lines from corridor

def get_lines(corridor: Mapping[str, Mapping],
              path: Union[List, None]=None
              ) -> List[List[str]]:
    """
    Get all the lines in the corridor
    
    Args:
        corridor (dict): dictionary with the corridor structure
        path (list, optional): list of nodes
    
    Returns:
        list of lines
    """
    if path is None:
        path = []

    lines = []
    for node, child in corridor.items():
        new_path = path + [node]
        if not child:  # If the node has no children, it is a leaf
            lines.append(new_path)
        else:
            lines.extend(get_lines(child, new_path))  # If the node has children, we call the function recursively

    return lines

get_lines(corridor)

[['MAD', 'CIU', 'COR', 'SEV', 'CAD'],
 ['MAD', 'CIU', 'COR', 'PGE', 'ANT', 'GRA'],
 ['MAD', 'CIU', 'COR', 'PGE', 'ANT', 'MAL']]

### 1.2. Sample a random line, and a random route from the line

In [4]:
def sample_line(lines: list) -> list:
    """
    Sample a random line from the list of lines
    
    Args:
        lines (list): list of lines
    
    Returns:
        list: random line
    """
    return lines[np.random.randint(len(lines))] 

def sample_route(line: list) -> list:
    """
    Sample a random route from line
    
    Args:
        line (list): list of stations
        
    Returns:
        list: random route
    """
    return line[np.random.randint(0, len(line)-1):]

lines = get_lines(corridor)

line = sample_line(lines)
print(f"Sampled line: {line}")

# Sample a random route from line (at least two stations)
route = sample_route(line)
print(f"Sampled route: {route}")

Sampled line: ['MAD', 'CIU', 'COR', 'PGE', 'ANT', 'GRA']
Sampled route: ['PGE', 'ANT', 'GRA']


### 1.3 Generate random timetable for a route

- Times in minutes.
- Initial time is randomized between 0 and 24*60 (24 hours).
- The time between stations is also randomized between 30 and 120 minutes.
- The time to dwell at each station is randomized between 2 and 8 minutes.

In [5]:
def get_timetable(route: list) -> dict:
    """
    Generate random timetable for route r
    
    Args:
        route (list): list of stations
    
    Returns:
        dict: timetable
    """
    timetable = {}
    AT = np.random.randint(0, 24*60)
    DT = AT
    for i, sta in enumerate(route):
        if i == 0 or i == len(route)-1:
            timetable[sta] = (AT, AT)
        else:
            timetable[sta] = (AT, DT)
            
        AT += np.random.randint(30, 120)
        DT = AT + np.random.randint(2, 8)
        
    return timetable

get_timetable(route)

{'PGE': (1292, 1292), 'ANT': (1343, 1348), 'GRA': (1386, 1386)}

## 2. Generate as many requested services as needed

In [6]:
# Generate random requested timetable in corridor for a day t

def get_schedule_request(n_services: int) -> dict:
    """
    Generate random timetable
    
    Args:
        n_services (int): number of services
    
    Returns:
        dict: timetable
    """
    return {i: get_timetable(sample_route(sample_line(lines))) for i in range(1, n_services+1)}

# Generate random schedule
schedule = get_schedule_request(3)
schedule

{1: {'PGE': (174, 174), 'ANT': (251, 254), 'MAL': (342, 342)},
 2: {'PGE': (169, 169), 'ANT': (255, 261), 'GRA': (357, 357)},
 3: {'ANT': (189, 189), 'GRA': (253, 253)}}

## 3. Define feasibility function

In [7]:
schedule = get_schedule_request(3)
schedule

{1: {'PGE': (1200, 1200), 'ANT': (1318, 1321), 'GRA': (1397, 1397)},
 2: {'CIU': (302, 302),
  'COR': (337, 343),
  'SEV': (369, 373),
  'CAD': (478, 478)},
 3: {'MAD': (1278, 1278),
  'CIU': (1323, 1328),
  'COR': (1373, 1377),
  'PGE': (1470, 1474),
  'ANT': (1546, 1549),
  'GRA': (1616, 1616)}}

In [8]:
# Dummy schedule
schedule = {1: {'MAD': (0, 0), 'BAR': (148, 152), 'FIG': (180, 180)},
            2: {'MAD': (8, 8), 'ZAR': (28, 30), 'BAR': (165, 167), 'FIG': (210, 210)},
            3: {'MAD': (30, 30), 'BAR': (180, 182), 'FIG': (225, 225)}}

In [9]:
from scipy.stats import loguniform

np.random.seed(seed=22)

def get_revenue_behaviour(schedule: dict) -> dict:
    """
    Get revenue behaviour
    
    Args:
        schedule (dict): schedule
    
    Returns:
        dict: revenue behaviour
    """
    revenue = {}
    bias = [0.2, 0.35, 0.1]
    for service in schedule:
        b = np.random.choice(bias)
        base_price = 55 * len(schedule[service])
        canon = base_price + b * base_price
        k = loguniform.rvs(0.01, 100, 1)
        max_penalty = canon * 0.4
        dt_penalty = max_penalty * 0.35
        tt_penalty = (max_penalty - dt_penalty) / (len(schedule[service]) - 1)
        revenue[service] = {'canon': canon, 'k': k, 'dt_max_penalty': dt_penalty, 'tt_max_penalty': tt_penalty}
    return revenue

revenue = get_revenue_behaviour(schedule)
revenue

{1: {'canon': 222.75,
  'k': 7.424686268142557,
  'dt_max_penalty': 31.185000000000002,
  'tt_max_penalty': 28.957500000000003},
 2: {'canon': 264.0,
  'k': 1.4810078247291318,
  'dt_max_penalty': 36.96,
  'tt_max_penalty': 22.880000000000006},
 3: {'canon': 181.5,
  'k': 14.186717826602765,
  'dt_max_penalty': 25.41,
  'tt_max_penalty': 23.595000000000006}}

In [10]:
sm = RevenueMaximization(schedule, revenue, safe_headway=10)

In [11]:
gsa_algo = GSA(objective_function=sm.get_fitness_gsa,
               is_feasible=sm.feasible_services_times,
               r_dim=len(sm.real_boundaries),
               d_dim=0,
               boundaries=sm.boundaries)

In [12]:
import time

gsa_algo.set_seed(seed=28)

pop_size = 5

start = time.time()
training_history = gsa_algo.optimize(population_size=pop_size,
                                     iters=100,
                                     chaotic_constant=True,
                                     repair_solution=True,
                                     initial_population=sm.get_initial_population(pop_size))

print(f"Elapsed time: {time.time() - start}")

Initial population: [<src.entities.Solution object at 0x13b3e0e20>, <src.entities.Solution object at 0x13ae1ca30>, <src.entities.Solution object at 0x13b3e9d00>, <src.entities.Solution object at 0x13b3e9e50>, <src.entities.Solution object at 0x1032b1730>]
GSA is optimizing  "get_fitness_gsa"
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 1 the best fitness is 517.0149775314684']
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 2 the best fitness is 530.855182872092']
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 3 the best fitness is 542.6469392432284']
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 4 the best fitness is 545.4742591617611']
Repairing solution...
Repairing solution...
Repairing solution...
Repairing solution...
['At iteration 5 the best fitness is 546

In [18]:
training_history

,Iteration,Fitness,Accuracy,ExecutionTime,Discrete,Real
0,0,517.014978,0,0.002603,[],"[-4.98, 157.83, 161.92, 189.92, 11.98, 35.76, ..."
1,1,530.855183,0,0.083370,[],"[-2.7546657683245406, 157.40093372659027, 161...."
2,2,542.646939,0,0.159620,[],"[-1.4908902847345742, 157.03956686173672, 161...."
3,3,545.474259,0,0.220369,[],"[-1.224344543318245, 156.72972567458024, 161.2..."
4,4,546.566945,0,0.298884,[],"[-1.0640064070495565, 156.6703265272249, 161.1..."
...,...,...,...,...,...,...
95,95,581.075606,0,7.828884,[],"[-0.537291897413701, 154.53825096383756, 158.9..."
96,96,581.075606,0,7.933295,[],"[-0.537291897413701, 154.53825096383756, 158.9..."
97,97,581.075606,0,8.038916,[],"[-0.537291897413701, 154.53825096383756, 158.9..."
98,98,581.075606,0,8.143058,[],"[-0.537291897413701, 154.53825096383756, 158.9..."


In [16]:
# Get last value in column 'Real' of training_history
best_solution = training_history.iloc[-1]['Real']
best_solution

array([ -0.5372919 , 154.53825096, 158.98644401, 188.47959778,
        10.72209259,  31.78793484,  33.76004547, 174.41483638,
       176.74351009, 219.95012524,  37.48580175, 186.57630266,
       190.67360173, 233.59831191])

In [22]:
sm.update_feasible_schedules(Solution(real=np.array(sm.requested_times, dtype=float), discrete=np.array([])))
sm.feasible_schedules

[[0, 0, 0], [0, 0, 1], [0, 1, 0], [0, 1, 1], [1, 0, 0], [1, 0, 1]]

In [14]:
S_i = sm.get_best_schedule(Solution(real=best_solution, discrete=np.array([])))
print(S_i)

sm.get_revenue(Solution(real=best_solution, discrete=S_i))

[1 0 0]


221.87179680826108

In [52]:
sm.get_best_schedule(Solution(real=best_solution, discrete=np.array([])))

array([1, 0, 0])

In [53]:
sum([revenue[service]['canon'] for service in revenue])

668.25

In [30]:
training_history

,Iteration,Fitness,Accuracy,ExecutionTime,Discrete,Real
0,0,517.014978,0,0.002766,[],"[-10.0, 156.30982941209513, 160.30982941209567..."
1,1,530.855183,0,0.072289,[],"[-10.0, 156.30982941209513, 160.30982941209567..."
2,2,531.910192,0,0.150944,[],"[-10.0, 156.30982941209513, 160.30982941209567..."
3,3,553.523532,0,0.227661,[],"[-10.0, 156.30982941209513, 160.30982941209567..."
4,4,553.656856,0,0.305343,[],"[-10.0, 156.30982941209513, 160.30982941209567..."
...,...,...,...,...,...,...
295,295,591.296411,0,12.379785,[],"[-10.0, 156.30982941209507, 160.30982941209567..."
296,296,591.296411,0,12.381563,[],"[-10.0, 156.30982941209507, 160.30982941209567..."
297,297,591.296411,0,12.383364,[],"[-10.0, 156.30982941209507, 160.30982941209567..."
298,298,591.296411,0,12.385157,[],"[-10.0, 156.30982941209507, 160.30982941209567..."
